In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import matplotlib.pyplot as plt
from ipyleaflet import Map, GeoJSON, GeoData, basemaps, LayersControl
from shapely.geometry import Point, Polygon, LineString
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
import rasterio
import rasterio.mask
from rasterio.plot import reshape_as_image 
from rasterio.plot import reshape_as_raster
from shapely.ops import unary_union

In [ ]:
# Opening JSON file
f = open('Rotterdam_data/historic_towages.json')
 
# returns JSON object as a dictionary
tow_data = json.load(f)
 
# Closing file
f.close()

In [ ]:
# # Create empty lists to store tugs data and their original JSON index
# tugs_data = []
# original_index = []

# # Extract tugs data into a separate list and store their original index
# for i, item in enumerate(tow_data):
#     for tug in item['tugs']:
#         tugs_data.append(tug)
#         original_index.append(i)

# # Create a DataFrame for tugs data with the original index
# tugs_df = pd.DataFrame(tugs_data)
# tugs_df['original_index'] = original_index  # Add original index as a new column


# # Create a DataFrame for the rest of the data
# vessel_data = [{'from': item['from'],
#                'to': item['to'],
#                'vessel': item['vessel'],
#                'type': item['type'],
#                'additional_data': item['additional_data']}
#               for item in tow_data]

# vessel_df = pd.DataFrame(vessel_data)

# # Display the DataFrames
# tugs_df

In [ ]:
# Create empty lists to store tugs data and their original JSON index
tugs_datab = []
original_indexb = []
vessel_name = []  # Create a list to store previous vessel names
vessel_mmsi = []  # Create a list to store previous vessel names


# Extract tugs data into a separate list, store their original index, and the previous vessel name
for i, item in enumerate(tow_data):
    for tug in item['tugs']:
        tugs_datab.append(tug)
        original_indexb.append(i)
        vessel_name.append(item['vessel']['name'])
        vessel_mmsi.append(item['vessel']['mmsi'])


# Create a DataFrame for tugs data with the original index and previous vessel name
tugs_dfb = pd.DataFrame(tugs_datab)
tugs_dfb['original_index'] = original_indexb  # Add original index as a new column
tugs_dfb['vessel_name'] = vessel_name  # Add vessel name as a new column
tugs_dfb['vessel_mmsi'] = vessel_mmsi


# Create a DataFrame for the rest of the data (excluding tugs)
vessel_datab = [{'from': item['from'],
               'to': item['to'],
               'vessel': item['vessel'],
               'type': item['type'],
               'additional_data': item['additional_data']}
              for item in tow_data]

vessel_dfb = pd.DataFrame(vessel_datab)


In [ ]:
tugs_dfb = tugs_dfb.merge(vessel_dfb['type'], left_on='original_index', right_index=True)

In [ ]:
tugs_incoming = tugs_dfb[tugs_dfb['type']=='incoming']
tugs_incoming = tugs_incoming[['from',	'to',	'from_location' ,'to_haven', 'original_index']]

# Create a Shapely Point geometry from the "from_location" coordinates
tugs_incoming['geometry'] = tugs_incoming['from_location'].apply(lambda coord: Point(coord))

# Convert the DataFrame to a GeoDataFrame
tugs_incoming_gdf = gpd.GeoDataFrame(tugs_incoming, geometry='geometry')
tugs_incoming_gdf = tugs_incoming_gdf.set_crs("EPSG:4326")

In [ ]:
tugs_dfb[['from_long', 'from_lat']] = tugs_dfb['from_location'].apply(lambda x: pd.Series(str(x).strip('[]').split(','))).astype(float)
tugs_dfb[['to_long', 'to_lat']] = tugs_dfb['to_location'].apply(lambda x: pd.Series(str(x).strip('[]').split(','))).astype(float)
tugs_dfb

Import other info

In [ ]:
def flatten_dict(d, parent_key='', sep='_'):
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

# Opening JSON file
f = open('Rotterdam_data/ais_rotterdam/ais_rotterdam_6.json')
 
# returns JSON object as a dictionary
data = json.load(f)
 
# Flatten each dictionary and store the results in a list
flattened_dicts = [flatten_dict(d) for d in data['data']]

# Create a DataFrame
data = pd.DataFrame(flattened_dicts)

# Closing file
f.close()

In [ ]:
# #data['navigation_destination_eta'] = pd.to_datetime(data['navigation_destination_eta'])
# #data['navigation_time'] = pd.to_datetime(data['navigation_time'])
# # Remove rows where 'vessel_type' is 'tug'
# data = data[data['vessel_type'] != 'tug']
# # Remove the 'vessel_subtype' column
# data = data.drop('vessel_subtype', 'vessel_callsign','navigation_status', 'navigation_destination_eta', 'navigation_heading', 'navigation_time', 'navigation_course', 'navigation_speed	', 'navigation_location_type', 'navigation_location_coordinates')
# #data[['navigation_location_lat', 'navigation_location_long']] = data['navigation_location_coordinates'].apply(lambda x: pd.Series(str(x).strip('[]').split(','))).astype(float)
# data

In [ ]:
data.head(10)

In [ ]:
columns_to_drop = [
    'vessel_callsign',
    'navigation_heading',
    'navigation_course',
    'navigation_speed',
    'navigation_location_type',
]

numeric_columns = [
    'device_mmsi',
    'device_dimensions_to_bow',           
    'device_dimensions_to_stern',       
    'device_dimensions_to_starboard',     
    'device_dimensions_to_port',
    #'vessel_imo'
    'navigation_draught',
    #'vessel_type',
    #'vessel_subtype'
]

info_columns = [
    'device_mmsi',
    'vessel_type',
    #'vessel_subtype'
]


In [ ]:
vessel_size = data[numeric_columns]
vessel_size = vessel_size.groupby('device_mmsi').median()
vessel_size['length'] = vessel_size['device_dimensions_to_bow'] + vessel_size['device_dimensions_to_stern']
vessel_size['width'] = vessel_size['device_dimensions_to_starboard'] + vessel_size['device_dimensions_to_port']
vessel_size.drop(['device_dimensions_to_bow','device_dimensions_to_stern', 'device_dimensions_to_starboard' ,'device_dimensions_to_port'], axis=1, inplace=True)
vessel_size.reset_index(inplace=True)


In [ ]:
vessel_info = data[info_columns]

vessel_info = vessel_info.groupby('device_mmsi').agg(lambda x: x.mode(0))
vessel_info.reset_index(inplace=True)


In [ ]:
vessel_data = pd.merge(vessel_size,vessel_info)
vessel_data['vessel_type_Code'], unique_values = pd.factorize(vessel_data['vessel_type'])

In [ ]:
main_db = pd.merge(tugs_dfb,vessel_data,how= 'right',left_on='vessel_mmsi',right_on='device_mmsi')
main_db.dropna(inplace=True)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from geopy.distance import geodesic
import folium
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'column_name' is the name of the column containing strings in your DataFrame
label_encoder = LabelEncoder()
main_db['Incoming to'] = label_encoder.fit_transform(main_db['to_berth'])
main_db


For the "from"

In [ ]:
# Define categorical and numeric columns
#categorical_cols = ['vessel_type', 'vessel_callsign', 'vessel_subtype', 'vessel_name', 'navigation_status']
features = ['navigation_draught', 
            'length',	
            'width',  
            'vessel_type_Code', 
            'Incoming to']

X = main_db[features].values

# Assuming 'latitude' and 'longitude' are your target variables (y-values)
# Target variables 
y_from_lat = main_db['to_lat'].values
y_from_lon = main_db['to_long'].values


Linear Regression

In [ ]:
# Split data into training and testing sets for latitude prediction
X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(X, y_from_lat, test_size=0.2, random_state=42)

# Initialize and train the linear regression model for latitude
linear_model_lat = LinearRegression()
linear_model_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = linear_model_lat.predict(X_from_lat_test)

# Calculate mean squared error for latitude prediction
mse_lat = mean_squared_error(y_from_lat_test, predictions_lat)
print(f"Linear Regression Mean Squared Error (Latitude): {mse_lat}")

# Split data into training and testing sets for longitude prediction
X_from_lon_train, X_from_lon_test, y_from_lon_train, y_from_lon_test = train_test_split(X, y_from_lon, test_size=0.2, random_state=42)

# Initialize and train the linear regression model for longitude
linear_model_lon = LinearRegression()
linear_model_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = linear_model_lon.predict(X_from_lon_test)

# Calculate mean squared error for longitude prediction
mse_lon = mean_squared_error(y_from_lon_test, predictions_lon)
print(f"Linear Regression Mean Squared Error (Longitude): {mse_lon}")

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Calculate average geodesic distance
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")

# Create a map centered at a specific location (e.g., mean latitude and longitude of your data)
# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
a = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(a)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(a)

# Save the map as an HTML files
a.save('map_linear_regression.html')



In [ ]:
from IPython.display import display

# Your previous code for creating the map
display(a)

Decision Tree

In [ ]:
# Initialize and train the decision tree regression model for latitude
decision_tree_lat = DecisionTreeRegressor(random_state=42)
decision_tree_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = decision_tree_lat.predict(X_from_lat_test)

# Calculate mean squared error for latitude prediction
mse_lat = mean_squared_error(y_from_lat_test, predictions_lat)
print(f"Decision Tree Mean Squared Error (Latitude): {mse_lat}")

# Initialize and train the decision tree regression model for longitude
decision_tree_lon = DecisionTreeRegressor(random_state=42)
decision_tree_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = decision_tree_lon.predict(X_from_lon_test)

# Calculate mean squared error for longitude prediction
mse_lon = mean_squared_error(y_from_lon_test, predictions_lon)
print(f"Decision Tree Mean Squared Error (Longitude): {mse_lon}")

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Calculate average geodesic distance
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")

# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
b = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(b)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(b)

# Save the map as an HTML file
b.save('map_decision_tree.html')
display(b)



Random Forest Regression Model

In [ ]:
# Initialize and train the random forest regression model for latitude
random_forest_lat = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = random_forest_lat.predict(X_from_lat_test)

# Calculate mean squared error for latitude prediction
mse_lat = mean_squared_error(y_from_lat_test, predictions_lat)
print(f"Random Forest Mean Squared Error (Latitude): {mse_lat}")

# Initialize and train the random forest regression model for longitude
random_forest_lon = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = random_forest_lon.predict(X_from_lon_test)

# Calculate mean squared error for longitude prediction
mse_lon = mean_squared_error(y_from_lon_test, predictions_lon)
print(f"Random Forest Mean Squared Error (Longitude): {mse_lon}")

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Calculate average geodesic distance
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")

# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
c = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(c)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(c)

# Save the map as an HTML file
c.save('map_random_forest.html')
display(c)



Gradient Boosting Regression Model

In [ ]:
# Initialize and train the gradient boosting regression model for latitude
gradient_boosting_lat = GradientBoostingRegressor(n_estimators=100, random_state=42)
gradient_boosting_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = gradient_boosting_lat.predict(X_from_lat_test)

# Initialize and train the gradient boosting regression model for longitude
gradient_boosting_lon = GradientBoostingRegressor(n_estimators=100, random_state=42)
gradient_boosting_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = gradient_boosting_lon.predict(X_from_lon_test)

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Calculate average geodesic distance
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")

# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
d = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):  # You can change this to display more or fewer points
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(d)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(d)

# Save the map as an HTML file
d.save('map_gradient_boosting.html')
display(d)


In [ ]:
from xgboost import XGBRegressor

# Initialize and train the XGBoost regression model for latitude
xgb_lat = XGBRegressor(n_estimators=100, random_state=42)
xgb_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = xgb_lat.predict(X_from_lat_test)

# Initialize and train the XGBoost regression model for longitude
xgb_lon = XGBRegressor(n_estimators=100, random_state=42)
xgb_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = xgb_lon.predict(X_from_lon_test)

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Calculate average geodesic distance
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")

# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
xgb_map = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):  # You can change this to display more or fewer points
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(xgb_map)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(xgb_map)

# Save the map as an HTML file
xgb_map.save('map_xgboost.html')
display(xgb_map)


In [ ]:
from sklearn.cluster import KMeans
import folium

# Create a KMeans clusterer for latitude and longitude
kmeans = KMeans(n_clusters=3, random_state=42)  # Change the number of clusters as needed
clusters = kmeans.fit_predict(X)

# Add cluster labels to the original dataframe
data['cluster'] = clusters

# Define the latitude and longitude to center the map
center_latitude = data['Latitude'].mean()
center_longitude = data['Longitude'].mean()

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
kmeans_map = folium.Map(location=map_center, zoom_start=12)

# Add clustered points to the map
for cluster in set(clusters):
    cluster_data = data[data['cluster'] == cluster]
    for index, row in cluster_data.iterrows():
        folium.Marker(location=[row['Latitude'], row['Longitude']], popup=f'Cluster {cluster}', icon=folium.Icon(color='blue')).add_to(kmeans_map)

# Save the map as an HTML file
kmeans_map.save('map_kmeans.html')
display(kmeans_map)

In [ ]:
from sklearn.cluster import DBSCAN

# Create a DBScan clusterer for latitude and longitude
dbscan = DBSCAN(eps=0.1, min_samples=5)
clusters = dbscan.fit_predict(X)

# Add cluster labels to the original dataframe
data['cluster'] = clusters

# Define the latitude and longitude to center the map
center_latitude = data['Latitude'].mean()
center_longitude = data['Longitude'].mean()

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
dbscan_map = folium.Map(location=map_center, zoom_start=12)

# Add clustered points to the map
for cluster in set(clusters):
    cluster_data = data[data['cluster'] == cluster]
    for index, row in cluster_data.iterrows():
        folium.Marker(location=[row['Latitude'], row['Longitude']], popup=f'Cluster {cluster}', icon=folium.Icon(color='blue')).add_to(dbscan_map)

# Save the map as an HTML file
dbscan_map.save('map_dbscan.html')
display(dbscan_map)
